# Projeto 2 - Ciência dos Dados

Nome:Luís Filipe Loureiro

Nome: João Bueno


___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [ ]:
%%capture

#Instalando o tweepy
# !pip install tweepy

In [1]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np

___
## Autenticando no  Twitter

* Conta: @luisfloureiro1

In [ ]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @luisfloureiro1

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [ ]:
#Produto escolhido:
produto = 'Golf'

#Quantidade mínima de mensagens capturadas:
n = 1600
#Quantidade mínima de mensagens para a base de treinamento:
t = 500

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []

for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break


#Embaralhando as mensagens para reduzir um possível viés
shuffle(list(set(msgs)))



Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(list(set(msgs))[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(list(set(msgs))[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [2]:
#Para não ser mais necessário conectar-se ao twitter, basta rodas essa cédula e a cédula que importa as bibliotecas
tweets= pd.read_excel('Golf.xlsx')


In [3]:
valores=[]

for valor in tweets.loc[:,"Relevancia"]:
    valores.append(valor)


In [14]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;/@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [15]:
clean_treinamento=[]
for tweet in tweets.loc[:,"Treinamento"]:
    clean_treinamento.append((cleanup(tweet)))

    
        
# clean_treinamento

In [16]:
tweets_clean =pd.DataFrame(clean_treinamento)
tweets_clean.loc[:,"Relevancia"]=1

In [17]:
#Adicionando a coluna relevancia
for numero in np.arange(0,500):
    tweets_clean.loc[numero,"Relevancia"]=valores[numero]

tweets_clean.columns=["tweet","Relevancia"]


In [18]:
#Retirando double spaces
for numero in np.arange(0,500):
    tweets_clean.loc[numero,"tweet"]=re.sub(' +', ' ',tweets_clean.loc[numero,"tweet"])
    tweets_clean.loc[numero,"tweet"]=tweets_clean.loc[numero,"tweet"].strip()

tweets_clean

,tweet,Relevancia
0,ele ah nsql te vejo passar aqui na frente com ...,0
1,não vai ter eu dando rolê de golf lotado de so...,1
2,caixa de cambio vw golf gl 1 8 alemão completo...,1
3,queria jogar golf https t co lp7zesdest,0
4,rt hbarbeiro faísca apoia o fim do uso das alg...,0
5,corrupt af https t co oxorbbaegc,0
6,beatriizromao nome giovanna\nlugar geórgia \nb...,1
7,vai toma no cuu\no cara jogando golf no cu do ...,0
8,lubsin fantucigu fzd a marielisayuri namorar o...,1
9,curiosidades do golfe ⛳\n\no golfe é um esport...,0


___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**